<a href="https://colab.research.google.com/github/Seohyun29/HAWK-EYE-Project/blob/main/%EA%B2%BD%EA%B8%B0%EC%A0%95%EB%B3%B4_%ED%81%AC%EB%A1%A4%EB%9F%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KBO 경기정보 크롤러
https://www.koreabaseball.com/Record/Player/HitterBasic/Basic1.aspx

In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [ ]:
#코랩으로 할때 설치
!apt-get update
!apt-get install -y chromium-chromedriver

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [665 kB]
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,685 kB]
Get:13 http://archive.u

In [ ]:
# 라이브러리 불러오기
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.select import Select
from bs4 import BeautifulSoup
import random
import pandas as pd
import os

In [ ]:
#코랩에서 실행하는 경우

# 드라이버 연결(수정)
driver = webdriver.Chrome(options=chrome_options)   # webdriver 연결을 chrome으로 연결하기
# 액션체인 객체 생성
act = ActionChains(driver)    # ActionChains : 동작을 여러 개 묶어서 실행시켜주는 메소드. 괄호에 있는 드라이버를 통해 크롬과 연결

# 스트존 웹사이트 이동(game 뒤의 특정날짜 입력으로 경기일자 변경가능)
driver.get('https://strikes.zone/game/230725SSGSSL')  #driver.get = get 메소드로 strikes.zone으로 이동(날짜와 연관이 있어 다른 날짜를 쓰려면 바꿔줘야)

# 대기
driver.implicitly_wait(3)    # driver.implictly로 시간 3초 지연설정

# 이닝별 정보 버튼 클릭
btn_inning = driver.find_element(By.XPATH, '//*[@id="__layout"]/div/div/section/section/header/div[2]/button[4]') # drive.find로 클릭하기(XPATH로 웹페이지로 가야 할 길을 열어놓는~)
act.click(btn_inning) # act 액션 객체가 btn_innineg을 클릭하도록 실행
act.perform()    # act의 클릭 동작을 실행하기

NameError: name 'chrome_options' is not defined

## KBO 타자 기록실 연결

In [ ]:
def connect_KBO():
    global driver
    global act

    # 드라이버 연결
    driver = webdriver.Chrome()

    # 액션체인 객체 생성
    act = ActionChains(driver)

    # KBO 기록실 이동 - 타자 / 투수
    driver.get("https://www.koreabaseball.com/Schedule/Schedule.aspx")

    # 대기
    driver.implicitly_wait(3)

## 연도별 경기정보 수집기

In [ ]:
def get_games_info(year):
    games_info = list()         #최종 데이터를 넣어주기 위한 리스트 생성

    year_btn = driver.find_element(By.XPATH, '//*[@id="ddlYear"]')          #ddlYear의 XPath를 가져옴
    sel_year_btn = Select(year_btn)                                         #ddlYear 선택
    sel_year_btn.select_by_value(str(year))                                 #내가 입력한 년도를 str로 변경하여 ddlYear 선택
    driver.implicitly_wait(random.randrange(3, 7))                          #로딩까지 기다림

    for month in range(1, 13):                                              #달 선택 (1~12월)
        month_btn = driver.find_element(By.XPATH, '//*[@id="ddlMonth"]')    #ddlMonth의 XPATH를 가져옴
        sel_month_btn = Select(month_btn)                                   #ddmMonth를 1~12월까지 선택

        month = "0"+str(month) if month < 10 else str(month)                #만약 달이 한자리수라면 앞에 0을 붙여주고 아니면 그대로 문자열로 반환
        sel_month_btn.select_by_value(month)                                #선택된 달로하여 선택박스 선택
        driver.implicitly_wait(random.randrange(3, 7))                      #로딩까지 기다림

        table = driver.find_element(By.XPATH, '/html/body/form/div[3]/section/div/div/div[4]/table/tbody')      #원하는 피쳐만 가져오기위해 피쳐 밑 데이터만 불러옴
        rows = table.find_elements(By.TAG_NAME, "tr")                       #tr을 하나씩 가져옴

        # 데이터가 없는 월은 continue
        if len(rows) == 1:                                                  #데이터가 없을 때 '데이터가 없습니다'라는 한줄만 존재
            continue                                                        #데이터 긁어오는거 없이 지나감


        game_date = None                                                    #game_data=None

        for tr in rows:                                                     #rows가 가지고 있는 tr을 하나씩 가져옴                                                   #info라는 list 생성
            datas = tr.find_elements(By.TAG_NAME, "td")                     #tr속에 있는 td데이터 이름 가져옴

            for td in datas:                                                #dates 속에 저장되어 있는 td를 하나씩 꺼내옴
                if td.get_attribute("rowspan"):                             #경기 날짜 데이터가 rowspan을 가지고 있는데 하루에 여러 경기가 있어서 새롭게 rowspan이 갱신될때 저장한다.
                    game_date = str(year) + "-" + td.text[:5].replace(".", "-") #날짜를 형식에 맞춰서 '.'을 '-'으로 바꿔줌
                if ":" in td.text:                                          # td.text속에 :이 있다면
                    game_start_time = td.text                               #경기 시작시간임으로 이를 게임시작 피쳐에 넣어줌
                if "vs" in td.text:                                         #'vs'가 있다면
                    away_name, home_name = td.text.split("vs")              #vs를 기준으로 앞은 원정팀, 뒤는 홈팀으로 나눠줌
                    away_name = "".join([i for i in away_name if i.isnumeric() == False])   #만약 숫자가 있다면 글을 안넣고 문자만 넣어줌
                    home_name = "".join([i for i in home_name if i.isnumeric() == False])   #만약 숫자가 있다면 글을 안넣고 문자만 넣어줌

                if td == datas[-2]:                                         #tr 중 뒤에서 두번째가 구장명임으로 2번째이면
                    game_place = td.text                                    #구장에 TEXT를 넣어줌

            games_info.append([game_date, game_place, home_name, away_name, game_start_time])       #수집한 정보를 games_info에 한번에 넣어줌

    return games_info



## 크롤러 실행기

In [ ]:
def runner(year):
    # 필요한 데이터 피쳐 리스트
    col = ["game_date", "game_place", "home_name", "away_name", "game_start_time"]          #우리가 사용할 피쳐명만 모아둔 곳
    f_name = f"KBO경기정보_{year}.csv"                                                      #파일 이름 설정
    print(f"현재 경로: [{os.getcwd()}]에 [{f_name}] 로 저장예정. \n원치 않으면 실행을 중단하세요.")

    connect_KBO()                                                                           #타자 기록실 연결 함수 실행
    data = get_games_info(year)                                                             #입력한 날짜에 맞춰 함수 실행

    df = pd.DataFrame(data, columns=col)                                                    #가져온 데이터를 데이터 프레임으로 제작
    df.to_csv(f_name, index=False)                                                          #csv파일로 저장

    print(f"파일 저장 완료\n파일 저장 경로: {os.getcwd()}/{f_name}")                        #저장이 끝나면 내용 프린

    return df

In [ ]:
runner(2020)